In [85]:
from io import StringIO
import boto3
import pandas as pd
import gzip
import pyarrow as pa
import pyarrow.parquet as pq
import os
from tqdm import tqdm

In [86]:
s3 = boto3.resource('s3', 
    endpoint_url='http://minio:9000',
    aws_access_key_id='admin',
    aws_secret_access_key='password',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
BUCKET='datalake'
TCPH_CSV_FOLDER='tcph_csv/'
TCPH_PARQUET_FOLDER='tcph_parquet/'
bucket = s3.Bucket(name=BUCKET)

In [ ]:
for object in tqdm(bucket.objects.filter(Prefix=TCPH_CSV_FOLDER)):
    if object.size <= 0:
        continue
    fileName = os.path.splitext(object.key)[0].split("/",1)[1]
    s3_obj = s3.Object(bucket_name=BUCKET, key=object.key)
    obj = s3_obj.get()['Body']
    df = pd.read_csv(obj, compression='gzip',header=None, sep='|')
    table = pa.Table.from_pandas(df)
    buf = pa.BufferOutputStream()
    pq.write_table(table, buf)
    s3.Bucket(BUCKET).put_object(Key=TCPH_PARQUET_FOLDER+fileName, Body=buf.getvalue().to_pybytes()) 

8it [00:05,  1.44it/s]
